In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import warnings

sys.path.append('../src/')
warnings.filterwarnings("ignore")

from server import Server
from utils import plot_exp

### First experiment : impact of federated learning

In [ ]:
LR = 0.01
EPOCHS = 100
PATIENCE = 8
NR_TRAINING_ROUNDS = 100
BATCH_SIZE = 8
RANGE_NR_CLIENTS = [1, 5, 10]

In [ ]:
experiment_losses, experiment_accs = [], []
for nr_clients in RANGE_NR_CLIENTS:
    print(f"### Number of clients : {nr_clients} ###\n\n")
    server = Server(
        nr_clients=nr_clients,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='MED',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=False,
        max_grad_norm=None,
        epsilon=None,
        is_parallel=False,
        device=None,
        verbose='server')
    test_losses, test_accs = server.train(early=True, patience=PATIENCE)
    experiment_losses.append(test_losses)
    experiment_accs.append(test_accs)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [6, 6]
plt.rcParams['figure.dpi'] = 100

In [ ]:
names = [f'{i} clients' for i in RANGE_NR_CLIENTS]
plot_exp(experiment_losses, experiment_accs, names)

### Second experiment : impact of differential privacy

In [ ]:
NR_CLIENTS = 5
NR_TRAINING_ROUNDS = 200
EPOCHS = 5
PATIENCE = 20
LR = 0.01
BATCH_SIZE = 8

MAX_GRAD_NORM = 1000
NOISE_MULTIPLIER = 0.1

RANGE_EPSILON = [100]

In [ ]:
experiment_losses, experiment_accs = [], []
for epsilon in RANGE_EPSILON:
    print(f"### epsilon : {epsilon} ###\n\n")
    server = Server(
        nr_clients=NR_CLIENTS,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='MED',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=True,
        epsilon=epsilon,
        max_grad_norm=MAX_GRAD_NORM,
        noise_multiplier=NOISE_MULTIPLIER,
        is_parallel=True,
        device=None,
        verbose='server')

    test_losses, test_accs = server.train(early=False, patience=PATIENCE)
    experiment_losses.append(test_losses)
    experiment_accs.append(test_accs)

In [ ]:
names = [f'ε = {i}' for i in RANGE_EPSILON]
plot_exp(experiment_losses, experiment_accs, names)